# Experimental Features
Here are some examples of experimental features we're planning to add to our API. These wrappers abstract away some
parts of the protobuf structs, and provide easier access to geometries and `datetime`.

## First Code Example
This is an alternative to the original version [here](./README.md#first-code-example). This simplifies the setting of
the `observed` field and hides the use of more verbose protobuf classes for both time filter (`observed`) and spatial
filter (`intersects`).

In [1]:
import tempfile
from datetime import date

from shapely.wkt import loads as load_wkt_shapely

from IPython.display import Image, display
from epl.geometry import Point
from nsl.stac import enum, utils
from nsl.stac.experimental import NSLClientEx, StacRequestWrap

# the client package stubs out a little bit of the gRPC connection code 
# get a client interface to the gRPC channel. This client singleton is threadsafe
client = NSLClientEx()

# create our request. this interface allows us to set fields in our protobuf object
request = StacRequestWrap()

# our area of interest will be the coordinates of the UT Stadium in Austin Texas
# the order of coordinates here is longitude then latitude (x, y). The results of our query 
# will be returned only if they intersect this point geometry we've defined (other geometry 
# types besides points are supported)
#
# This string format, POINT(float, float) is the well-known-text geometry format:
# https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry
#
# the epsg # defines the WGS-84 elispsoid (`epsg=4326`) spatial reference 
# (the latitude longitude  spatial reference most commonly used)
#
# the epl.geometry Point class is an extension of shapely's Point class that supports
# the protobuf definitions we use with STAC. To extract a shapely geometry from it use
# the shapely_dump property
request.intersects = Point.import_wkt(wkt="POINT(-97.7323317 30.2830764)", epsg=4326)
# or using `shapely`
request.intersects = load_wkt_shapely("POINT(-97.7323317 30.2830764)")

# The `set_observed` method allows for making sql-like queries on the observed field and the
# LTE is an enum that means less than or equal to the value in the query field
#
# This Query is for data from August 25, 2019 UTC or earlier
request.set_observed(rel_type=enum.FilterRelationship.LTE, value=date(2019, 8, 25))

# search_one_ex method requests only one item be returned that meets the query filters in the StacRequestWrap
# the item returned is a wrapper of the protobuf message; StacItemWrap. search_one_ex, will only return the most
# recently observed results that matches the time filter and spatial filter
stac_item = client.search_one_ex(request)

# get the thumbnail asset from the assets map. The other option would be a Geotiff, 
# with asset key 'GEOTIFF_RGB'
asset_wrap = stac_item.get_asset(asset_type=enum.AssetType.THUMBNAIL)

print(asset_wrap)
# uncomment to display image
# with tempfile.TemporaryDirectory() as d:
#     filename = utils.download_asset(asset=asset, save_directory=d)
#     display(Image(filename=filename))

nsl client connecting to stac service at: api.nearspacelabs.net:9090

attempting NSL authentication against https://api.nearspacelabs.net
fetching new authorization in 60 minutes
href: "https://api.nearspacelabs.net/download/20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183518Z_746_POM1_ST2_P.png"
type: "image/png"
eo_bands: RGB
asset_type: THUMBNAIL
cloud_platform: GCP
bucket_manager: "Near Space Labs"
bucket_region: "us-central1"
bucket: "swiftera-processed-data"
object_path: "20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183518Z_746_POM1_ST2_P.png"
extension: .png
asset_key: THUMBNAIL_RGB


## Simple Query and the Makeup of a StacItem
The easiest query to construct is a `StacRequestWrap` constructor with no variables, and the next simplest, is the case
where we know the STAC item `id` that we want to search. If we already know the STAC `id` of an item, we can construct
the `StacRequestWrap` as follows:

In [2]:
from nsl.stac.experimental import NSLClientEx, StacRequestWrap

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

# create a request for a specific STAC item
request = StacRequestWrap(id='20190822T183518Z_746_POM1_ST2_P')

# for this request we might as well use the search one, as STAC ids ought to be unique
stac_item = client.search_one_ex(request)
print(stac_item)

id: "20190822T183518Z_746_POM1_ST2_P"
collection: "NSL_SCENE"
properties {
  type_url: "nearspacelabs.com/proto/st.protobuf.v1.NslDatast.protobuf.v1.NslData/st.protobuf.v1.NslData"
  value: "\n\340\014\n\03620190822T162258Z_TRAVIS_COUNTY\"\003 \352\0052\03520200702T102306Z_746_ST2_POM1:\03520190822T183518Z_746_POM1_ST2:\03520200702T101632Z_746_ST2_POM1:\03520200702T102302Z_746_ST2_POM1:\03520200702T102306Z_746_ST2_POM1B\03520190822T183518Z_746_POM1_ST2H\001R\374\n\n$\004\304{?\216\371\350=\376\377\306>\300\327\256\275\323rv?2\026*D3Qy6\177>\3675\000\000\200?\022\024\r+}\303\302\025\033;\362A\0353}\367\300%g\232\250@\022\024\r\026}\303\302\025\376?\362A\035\000\367\235@%\232\t\331?\022\024\r\351|\303\302\025\021A\362A\035M\370\033\301%g\016\226\277\022\024\r\201|\303\302\025\3709\362A\035\000\252\245@%\315\3547?\022\024\r\310|\303\302\025\245G\362A\035\232\315l\301%3\347\270\300\022\024\rq|\303\302\025\2149\362A\035\000\376o@%\000(\017@\022\024\rD|\303\302\025oD\362A\0353\323\302\301%\3

## Spatial Queries

### Query by Bounds
You can query for STAC items intersecting a bounding box of minx, miny, maxx, and maxy. An
[epsg](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset) code is required (4326 is the most common epsg
code used for longitude and latitude). Remember, this finds STAC items that intersect the bounds, **not** STAC items
contained by the bounds.

In [3]:
from nsl.stac.experimental import StacRequestWrap, NSLClientEx

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

# request wrapper
request = StacRequestWrap()

# define our area of interest bounds using the xmin, ymin, xmax, ymax coordinates of an area on 
# the WGS-84 ellipsoid
neighborhood_box = (-97.7352547645, 30.27526474757116, -97.7195692, 30.28532)
# setting the bounds tests for intersection (not contains)
request.set_bounds(neighborhood_box, epsg=4326)
request.limit = 3

# Search for data that intersects the bounding box
epsg_4326_ids = []
for stac_item in client_ex.search_ex(request):
    print("STAC item id: {}".format(stac_item.id))
    print("bounds:")
    print(stac_item.geometry.bounds)
    print("bbox (EnvelopeData protobuf):")
    print(stac_item.bbox)
    print("geometry:")
    print(stac_item.geometry)
    epsg_4326_ids.append(stac_item.id)

STAC item id: 20200703T174443Z_650_POM1_ST2_P
bounds:
(-97.74591162787891, 30.279053855446275, -97.7277773253055, 30.292199081479485)
bbox (EnvelopeData protobuf):
xmin: -97.74591162787891
ymin: 30.279053855446275
xmax: -97.7277773253055
ymax: 30.292199081479485
proj {
  epsg: 4326
}

geometry:
MULTIPOLYGON (((-97.74591162787891 30.28747759413035, -97.74220557686867 30.27905385544627, -97.7277773253055 30.28386159376494, -97.73145460028107 30.29219908147948, -97.74591162787891 30.28747759413035))) epsg: 4326

STAC item id: 20200703T174303Z_595_POM1_ST2_P
bounds:
(-97.75153797990234, 30.269721707638205, -97.73325611269058, 30.28300247580166)
bbox (EnvelopeData protobuf):
xmin: -97.75153797990234
ymin: 30.269721707638205
xmax: -97.73325611269058
ymax: 30.28300247580166
proj {
  epsg: 4326
}

geometry:
MULTIPOLYGON (((-97.75153797990234 30.27820420412512, -97.74797016158224 30.2697217076382, -97.73325611269058 30.27455757923618, -97.73689448438766 30.28300247580166, -97.75153797990234 30.

### Query by Bounds; Projection Support
Querying using geometries defined in a different projection requires defining the epsg number for the spatial reference
of the data. In this example we use an epsg code for a [UTM projection](https://epsg.io/3744).

Notice that the results below are the same as the cell above (look at the `TEST-->` section of the printout)

In [4]:
from nsl.stac.experimental import StacRequestWrap, NSLClientEx

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

# request wrapper
request = StacRequestWrap()

# define our area of interest bounds using the xmin, ymin, xmax, ymax coordinates of in UTM 14N in NAD83 (epsg 3744)
neighborhood_box = (621636.1875228449, 3349964.520449501, 623157.4212553708, 3351095.8075163467)
# setting the bounds tests for contains query
request.set_bounds(neighborhood_box, epsg=3744)
request.limit = 3

# Search for data that intersects the bounding box
for stac_item in client_ex.search_ex(request):
    print("STAC item id: {}".format(stac_item.id))
    print("bounds:")
    print(stac_item.geometry.bounds)
    print("bbox (EnvelopeData protobuf):")
    print(stac_item.bbox)
    print("geometry:")
    print(stac_item.geometry)
    print("TEST RESULT '{1}': stac_item id {0} from 3744 bounds is in the set of the 4326 bounds search results."
          .format(stac_item.id, stac_item.id in epsg_4326_ids))
    print()



STAC item id: 20200703T174443Z_650_POM1_ST2_P
bounds:
(-97.74591162787891, 30.279053855446275, -97.7277773253055, 30.292199081479485)
bbox (EnvelopeData protobuf):
xmin: -97.74591162787891
ymin: 30.279053855446275
xmax: -97.7277773253055
ymax: 30.292199081479485
proj {
  epsg: 4326
}

geometry:
MULTIPOLYGON (((-97.74591162787891 30.28747759413035, -97.74220557686867 30.27905385544627, -97.7277773253055 30.28386159376494, -97.73145460028107 30.29219908147948, -97.74591162787891 30.28747759413035))) epsg: 4326

TEST RESULT 'True': stac_item id 20200703T174443Z_650_POM1_ST2_P from 3744 bounds is in the set of the 4326 bounds search results.

STAC item id: 20200703T174303Z_595_POM1_ST2_P
bounds:
(-97.75153797990234, 30.269721707638205, -97.73325611269058, 30.28300247580166)
bbox (EnvelopeData protobuf):
xmin: -97.75153797990234
ymin: 30.269721707638205
xmax: -97.73325611269058
ymax: 30.28300247580166
proj {
  epsg: 4326
}

geometry:
MULTIPOLYGON (((-97.75153797990234 30.27820420412512, -97

### Query by GeoJSON
Use a GeoJSON geometry to define the `intersects` property.

In [5]:
import json
import requests
from epl.geometry import shape as epl_shape

from nsl.stac.experimental import StacRequestWrap, NSLClientEx

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

request = StacRequestWrap()

# retrieve a coarse geojson foot print of Travis County, Texas
r = requests.get("http://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/TX/Travis.geo.json")
travis_shape = epl_shape(r.json()['features'][0]['geometry'], epsg=4326)

# search for any data that intersects the travis county geometry
request.intersects = travis_shape

# limit results to 2 (instead of default of 10)
request.limit = 2

geojson_ids = []
# get a client interface to the gRPC channel
for stac_item in client_ex.search_ex(request):
    print("STAC item id: {}".format(stac_item.id))
    print("Stac item observed: {}".format(stac_item.observed))
    geojson_ids.append(stac_item.id)

STAC item id: 20201001T211834Z_2012_POM1_ST2_P
Stac item observed: 2020-10-01 21:18:34+00:00
STAC item id: 20201001T211832Z_2011_POM1_ST2_P
Stac item observed: 2020-10-01 21:18:32+00:00


### Query by WKT
Use a [WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) geometry to define the
`intersects` property.

In [6]:
from epl.geometry import Polygon

# Same geometry as above, but a wkt geometry instead of a geojson
travis_wkt = "POLYGON((-97.9736 30.6251, -97.9188 30.6032, -97.9243 30.5703, -97.8695 30.5484, \
              -97.8476 30.4717, -97.7764 30.4279, -97.5793 30.4991, -97.3711 30.4170, \
              -97.4916 30.2089, -97.6505 30.0719, -97.6669 30.0665, -97.7107 30.0226, \
              -98.1708 30.3567, -98.1270 30.4279, -98.0503 30.6251, -97.9736 30.6251))" 
request.intersects = Polygon.import_wkt(wkt=travis_wkt, epsg=4326)
request.limit = 2
for stac_item in client_ex.search_ex(request):
    print("STAC item id: {0} from wkt filter intersects result from geojson filter: {1}"
          .format(stac_item.id, stac_item.id in geojson_ids))

STAC item id: 20201001T211834Z_2012_POM1_ST2_P from wkt filter intersects result from geojson filter: True
STAC item id: 20201001T211832Z_2011_POM1_ST2_P from wkt filter intersects result from geojson filter: True


## Temporal Queries
When it comes to Temporal queries there are a few things to note. 

- we assume all dates are UTC unless specified
- datetime and date are treated differently in different situations
  - date for EQ and NEQ is a 24 hour period
  - datetime for EQ and NEQ is almost useless (it is a timestamp accurate down to the nanosecond)
  - date for GTE or LT is defined as the date from the first nanosecond of that date
  - date for LTE or GT is defined as the date at the final nanosecond of that date
  - date for start and end with BETWEEN has a start with minimum time and an end with the max time. same for NOT_BETWEEN
  - datetime for GTE, GT, LTE, LT, BETWEEN and NOT_BETWEEN is interpreted strictly according to the nanosecond of that
  datetime definition(s) provided

When creating a time query filter, we want to use the >, >=, <, <=, ==, != operations and inclusive and exclusive range
requests. We do this by using the `set_observed` method and the `value` set to a date/datetime combined with `rel_typ`
set to `GTE`,`GT`, `LTE`, `LT`, `EQ`, or `NEQ`. If we use `rel_type` set to `BETWEEN` OR `NOT_BETWEEN` then we must set
the `start` **and** the `end` variables.

### Everything After A Specific Date

In [7]:
from datetime import date, datetime
from nsl.stac.experimental import NSLClientEx, StacRequestWrap
from nsl.stac import enum

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

request = StacRequestWrap()

# make a filter that selects all data on or after August 21st, 2019
request.set_observed(rel_type=enum.FilterRelationship.GTE, value=date(2019, 8, 21))
request.limit = 2

demonstration_datetime = datetime.combine(date(2019, 8, 21), datetime.min.time())

for stac_item in client_ex.search_ex(request):
    print("STAC item date, {0}, is after {1}: {2}".format(
        stac_item.observed,
        demonstration_datetime,
        datetime.timestamp(stac_item.observed) > datetime.timestamp(demonstration_datetime)))

STAC item date, 2021-02-07 20:29:00+00:00, is after 2019-08-21 00:00:00: True
STAC item date, 2021-02-07 20:28:58+00:00, is after 2019-08-21 00:00:00: True


#### Everything Between Two Dates

Now we're going to do a range request and select data between two dates using the `start` and `end` parameters instead
of the `value` parameter:

In [8]:
from datetime import datetime, timezone, timedelta
from nsl.stac import enum

# Query data from August 1, 2019
start = datetime(2019, 8, 1, 0, 0, 0, tzinfo=timezone.utc)
# ... up until August 10, 2019
end = start + timedelta(days=9)

request.set_observed(rel_type=enum.FilterRelationship.BETWEEN, start=start, end=end)
request.limit = 2

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

for stac_item in client_ex.search_ex(request):
    print("STAC item date, {0}, is between {1} and {2}".format(
        stac_item.observed,
        datetime.combine(start, datetime.min.time()),
        datetime.combine(end, datetime.min.time())))

STAC item date, 2019-08-06 20:42:53+00:00, is between 2019-08-01 00:00:00 and 2019-08-10 00:00:00
STAC item date, 2019-08-06 20:42:51+00:00, is between 2019-08-01 00:00:00 and 2019-08-10 00:00:00


In the above print out we are returned STAC items that are between the dates of Aug 1 2019 and Aug 10 2019. Also, notice
there's no warnings as we defined our utc timezone on the datetime objects.

#### Select Data for One Day

Now we'll search for everything on a specific day using a python `datetime.date` for the `value` and `rel_type` set to
use equals (`FilterRelationship.EQ`). Python's `datetime.datetime` is a specific value and if you use it combined with
`EQ` the query would insist that the time relationship match down to the second. But since `datetime.date` is only
specific down to the day, the filter is created for the entire day. This will check for everything from the start until
the end of the 8th of August, specifically in the Austin, Texas timezone (UTC -6).

In [9]:
from datetime import datetime, timezone, timedelta, date
from nsl.stac.experimental import NSLClientEx, StacItemWrap
from nsl.stac import utils, enum

request = StacRequestWrap()

# Query all data for the entire day of August 6, 2019 (in TX time)
texas_utc_offset = timezone(timedelta(hours=-6))
request.set_observed(value=date(2019, 8, 6), rel_type=enum.FilterRelationship.EQ, tzinfo=texas_utc_offset)

request.limit = 2

# get a client interface to the gRPC channel
client_ex = NSLClientEx()
for stac_item in client_ex.search_ex(request):
    print(datetime.fromtimestamp(stac_item.observed.timestamp(), tz=timezone.utc))

2019-08-06 20:42:53+00:00
2019-08-06 20:42:51+00:00


## Query by GSD (ground sampling distance)

Our sample imagery contains both 30cm (0.15-0.20m gsd) and 10cm (0.05m gsd) scenes. Here is a snippet below detailing how to
filter by GSD.

In [ ]:
from itertools import islice

from nsl.stac.experimental import NSLClientEx
from nsl.stac import enum

request = StacRequestWrap()

# Query for all imagery whose GSD is less than 0.051 (to account for minor floating point variations)
request.set_gsd(rel_type=enum.FilterRelationship.LTE, value=0.051)

# Restrict search to areas of publicly available 10cm imagery in Austin, TX
request.intersects = Polygon.import_wkt(epsg=4326, wkt='POLYGON((-97.7344436645508 30.326004681223694,'
                                                       '-97.7344436645508 30.29517998555933,'
                                                       '-97.73650360107423 30.28569351372147,'
                                                       '-97.73616027832033 30.273834133861683,'
                                                       '-97.71796417236328 30.274130635825372,'
                                                       '-97.71796417236328 30.28539704669346,'
                                                       '-97.68397521972658 30.293994226755345,'
                                                       '-97.68466186523439 30.32630102546601,'
                                                       '-97.7344436645508 30.326004681223694))')

client_ex = NSLClientEx()

# filter results by the first we can download with basic credentials
items = islice(filter(lambda item: 'REGION_0' in item.get_asset(asset_type=enum.AssetType.GEOTIFF).href,
                      client_ex.search_ex(request, auto_paginate=True)), 1)

stac_item = next(items)
print(stac_item.id)
print(stac_item.gsd)

## Downloading from AssetWrap

In [10]:
import os
import tempfile
from IPython.display import Image, display

from epl.geometry import LineString
from nsl.stac.experimental import NSLClientEx, StacRequestWrap
from nsl.stac import utils, enum

request = StacRequestWrap()
request.intersects = LineString.import_wkt(wkt='LINESTRING(622301.8284206488 3350344.236542711,'
                                               '622973.3950196661 3350466.792693002)',
                                           epsg=3744)
request.set_observed(value=date(2019, 8, 25), rel_type=enum.FilterRelationship.LTE)
request.limit = 3

client_ex = NSLClientEx()

stac_item = client_ex.search_one_ex(request)
# get the thumbnail asset from the assets map
asset_wrap = stac_item.get_asset(asset_type=enum.AssetType.THUMBNAIL)
print(asset_wrap)

# (side-note delete=False in NamedTemporaryFile is only required for windows.)
with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as file_obj:
    asset_wrap.download(file_obj=file_obj)
    print("downloaded file {}".format(os.path.basename(asset_wrap.object_path)))
    print()
    # uncomment to display            
    # display(Image(filename=file_obj.name))

href: "https://api.nearspacelabs.net/download/20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183418Z_716_POM1_ST2_P.png"
type: "image/png"
eo_bands: RGB
asset_type: THUMBNAIL
cloud_platform: GCP
bucket_manager: "Near Space Labs"
bucket_region: "us-central1"
bucket: "swiftera-processed-data"
object_path: "20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183418Z_716_POM1_ST2_P.png"
extension: .png
asset_key: THUMBNAIL_RGB

downloaded file 20190822T183418Z_716_POM1_ST2_P.png

href: "https://api.nearspacelabs.net/download/20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183410Z_712_POM1_ST2_P.png"
type: "image/png"
eo_bands: RGB
asset_type: THUMBNAIL
cloud_platform: GCP
bucket_manager: "Near Space Labs"
bucket_region: "us-central1"
bucket: "swiftera-processed-data"
object_path: "20190822T162258Z_TRAVIS_COUNTY/Published/REGION_0/20190822T183410Z_712_POM1_ST2_P.png"
extension: .png
asset_key: THUMBNAIL_RGB

downloaded file 20190822T183410Z_712_POM1_ST2_P.png



## View
For our ground sampling distance query we're using another query filter; this time it's the
[FloatFilter](https://geo-grpc.github.io/api/#epl.protobuf.v1.FloatFilter). It behaves just as the TimestampFilter, but
with floats for `value` or for `start` + `end`.

In order to make our off nadir query we need to insert it inside of an
[ViewRequest](https://geo-grpc.github.io/api/#epl.protobuf.v1.ViewRequest) container and set that to the `view` field of
the `StacRequest`.

In [11]:
from datetime import datetime, timezone
from epl.geometry import Point
from nsl.stac.experimental import NSLClientEx, StacRequestWrap
from nsl.stac.enum import FilterRelationship, Mission

request = StacRequestWrap()

# create our off_nadir query to only return data captured with an angle of less than or 
# equal to 10 degrees
request.set_off_nadir(rel_type=FilterRelationship.GTE, value=30.0)

request.set_gsd(rel_type=FilterRelationship.LT, value=1.0)

# define ourselves a point in Texas
request.intersects = Point.import_wkt("POINT(621920.1090935947 3350833.389847579)", epsg=26914)
# the above could also be defined using longitude and latitude as follows:
# request.intersects = Point.import_wkt("POINT(-97.7323317 30.2830764)", epsg=4326)

# create a StacRequest with geometry, gsd, and off nadir and a limit of 4
request.limit = 4

# get a client interface to the gRPC channel
client_ex = NSLClientEx()
for stac_item in client_ex.search_ex(request):
    print("{0} STAC item '{1}' from {2}\nhas a off_nadir\t{3:.2f}, which should be greater than or "
          "equal to requested off_nadir\t{4:.3f} (confirmed {5})".format(
        stac_item.mission.name,
        stac_item.id,
        stac_item.observed,
        stac_item.off_nadir,
        request.stac_request.view.off_nadir.value,
        request.stac_request.view.off_nadir.value < stac_item.off_nadir))
    print("has a gsd\t{0:.3f}, which should be less than "
          "the requested\t\t  gsd\t\t{1:.3f} (confirmed {2})".format(
        stac_item.gsd,              
        request.stac_request.gsd.value,
        request.stac_request.gsd.value < stac_item.gsd))

SWIFT STAC item '20190806T202221Z_9007_POM1_ST2_P' from 2019-08-06 20:22:21+00:00
has a off_nadir	34.28, which should be greater than or equal to requested off_nadir	30.000 (confirmed True)
has a gsd	0.200, which should be less than the requested		  gsd		1.000 (confirmed False)
SWIFT STAC item '20190806T202219Z_9006_POM1_ST2_P' from 2019-08-06 20:22:19+00:00
has a off_nadir	34.51, which should be greater than or equal to requested off_nadir	30.000 (confirmed True)
has a gsd	0.200, which should be less than the requested		  gsd		1.000 (confirmed False)
SWIFT STAC item '20190806T202153Z_8993_POM1_ST2_P' from 2019-08-06 20:21:53+00:00
has a off_nadir	32.85, which should be greater than or equal to requested off_nadir	30.000 (confirmed True)
has a gsd	0.200, which should be less than the requested		  gsd		1.000 (confirmed False)
SWIFT STAC item '20190806T202151Z_8992_POM1_ST2_P' from 2019-08-06 20:21:51+00:00
has a off_nadir	33.23, which should be greater than or equal to requested off_nad

## Shapely Geometry

Both `StacItemWrap` and `StacRequestWrap` support `shapely` geometries by default, and can be used in place of
`epl.geometry` types.

In [12]:
from shapely.wkt import loads as loads_wkt

from nsl.stac.experimental import NSLClientEx, StacRequestWrap
from nsl.stac import utils, enum

request = StacRequestWrap()

request.intersects = loads_wkt('LINESTRING(-97.72842049283962 30.278624772098176,'
                               '-97.72142529172878 30.2796624743974)')

request.set_observed(value=date(2019, 8, 20), rel_type=enum.FilterRelationship.LTE)
request.limit = 10

client_ex = NSLClientEx()

unioned = None
for stac_item in client_ex.search_ex(request):
    if unioned is None:
        unioned = stac_item.geometry
    else:
        # execute shapely union
        unioned = unioned.union(stac_item.geometry)

print(unioned)

POLYGON ((-97.73904613302376 30.28558379365554, -97.7391983503974 30.2875173500651, -97.72321588821087 30.28827923391159, -97.72318191531373 30.28782344215122, -97.71713732528039 30.28831646331542, -97.71693109816546 30.28666272574028, -97.70874633971988 30.28734610398103, -97.70818071127752 30.28287053252838, -97.70808905472636 30.28287930690277, -97.70677365314802 30.27386748307901, -97.7170478085542 30.27277749017812, -97.71706056512183 30.27243547076341, -97.71909405701686 30.27256040213442, -97.7213917618061 30.27231663689927, -97.7211668184693 30.27047840774788, -97.73716286590115 30.26897383853585, -97.73753540641121 30.27221456038255, -97.74030883925472 30.27238987412984, -97.7401564450095 30.27643992828877, -97.74061099516257 30.27646759248412, -97.74006387853352 30.28564189160005, -97.73904613302376 30.28558379365554))


## Enum Classes
There are a number of different enum classes used for both STAC requests and STAC items.

In [13]:
from nsl.stac import enum
import inspect
[m for m in inspect.getmembers(enum) if not m[0].startswith('_') and m[0][0].isupper() and m[0] != 'IntFlag']

[('AssetType', <enum 'AssetType'>),
 ('Band', <enum 'Band'>),
 ('CloudPlatform', <enum 'CloudPlatform'>),
 ('Constellation', <enum 'Constellation'>),
 ('FilterRelationship', <enum 'FilterRelationship'>),
 ('Instrument', <enum 'Instrument'>),
 ('Mission', <enum 'Mission'>),
 ('Platform', <enum 'Platform'>),
 ('SortDirection', <enum 'SortDirection'>)]

In [14]:
# Specific to Queries
print("for defining the sort direction of a query")
for s in enum.SortDirection:
    print(s.name)
print("for defining the query relationship with a value (EQ, LTE, GTE, LT, GT, NEQ), a start-end range \n"
      "(BETWEEN, NOT_BETWEEN), a set (IN, NOT_IN) or a string (LIKE, NOT_LIKE). "
      "NOTE: EQ is the default relationship type")
for f in enum.FilterRelationship:
    print(f.name)

for defining the sort direction of a query
NOT_SORTED
DESC
ASC
for defining the query relationship with a value (EQ, LTE, GTE, LT, GT, NEQ), a start-end range 
(BETWEEN, NOT_BETWEEN), a set (IN, NOT_IN) or a string (LIKE, NOT_LIKE). To be noted, EQ is the default relationship type
EQ
LTE
GTE
LT
GT
BETWEEN
NOT_BETWEEN
NEQ
IN
NOT_IN
LIKE
NOT_LIKE


In [15]:
# Specific to Assets
print("these can be useful when getting a specific Asset from a STAC item by the type of Asset")
for a in enum.AssetType:
    print(a.name)

these can be useful when getting a specific Asset from a STAC item by the type of Asset
UNKNOWN_ASSET
JPEG
GEOTIFF
LERC
MRF
MRF_IDX
MRF_XML
CO_GEOTIFF
RAW
THUMBNAIL
TIFF
JPEG_2000
XML
TXT
PNG
OVERVIEW
JSON
HTML
WEBP


In [16]:
# STAC Item details
print("Mission, Platform and Instrument are aspects of data that can be used in queries.\n"
      "But as NSL currently only has one platform and one instrument, these may not be useful")
print("missions:")
for a in enum.Mission:
    print(a.name)
print("\nplatforms:")
for a in enum.Platform:
    print(a.name)
print("\ninstruments:")
for a in enum.Instrument:
    print(a.name)

Mission, Platform and Instrument are aspects of data that can be used in queries.
But as NSL currently only has one platform and one instrument, these may not be useful
missions:
UNKNOWN_MISSION
LANDSAT
NAIP
SWIFT
PNOA

platforms:
UNKNOWN_PLATFORM
LANDSAT_1
LANDSAT_2
LANDSAT_3
LANDSAT_123
LANDSAT_4
LANDSAT_5
LANDSAT_45
LANDSAT_7
LANDSAT_8
SWIFT_2

instruments:
UNKNOWN_INSTRUMENT
OLI
TIRS
OLI_TIRS
POM_1
TM
ETM
MSS
POM_2


## Subscriptions and Data Delivery
This feature is useful if you want to be subscribed to a `StacRequest`, receiving published assets to a cloud bucket.

### Cloud Accounts

    - AWS id: `275780812376`
    - GCP email: `thirdparty-bucket-access@swiftera-processed-data.iam.gserviceaccount.com`

### Example

Set up for the destination bucket is slightly different between AWS and GCP, so we'll review both.

Assume that we have an upcoming flight covering Austin, TX and you'd like only some STAC items to be copied to your 
bucket as they're published:

In [ ]:
from datetime import datetime, timezone

from nsl.stac import enum
from nsl.stac.destinations import AWSDestination, GCPDestination
from nsl.stac.experimental import StacRequestWrap, NSLClientEx

# get a client interface to the gRPC channel
client_ex = NSLClientEx()

# create a new request
request = StacRequestWrap()
# a polygon containing two University of Texas at Austin stadiums
request.intersects = Polygon.import_wkt('POLYGON((-97.73521184921263 30.28551192777313,'
                                        '-97.73522257804868 30.279369307168608,'
                                        '-97.72501945495603 30.279313716086122,'
                                        '-97.7251052856445 30.285530456966953,'
                                        '-97.73521184921263 30.28551192777313))',
                                        epsg=4326)
# we only want images delivered after the flight was conducted
request.set_observed(rel_type=enum.FilterRelationship.GTE, value=datetime(2021, 11, 1, tzinfo=timezone.utc))

### GCP Delivery

The destination bucket preparation process for GCP is fairly simple:

1. Create a GCP Role with only `storage.objectAdmin` permissions for your bucket (e.g. `my-bucket`).

2. Assign the newly created role (e.g. `NSLSubscription`) to our GCP service account,
`thirdparty-bucket-access@swiftera-processed-data.iam.gserviceaccount.com`. This can be done from the terminal, e.g.:

```bash
$ gsutil iam ch \
    serviceAccount:thirdparty-bucket-access@swiftera-processed-data.iam.gserviceaccount.com:roles/storage.objectAdmin \
    gs://my-bucket
```

Finally, lets create a `BaseDestination` object and finish creating our subscription:

In [ ]:
# creates a destination object to describe how assets should be delivered
dst = GCPDestination(asset_type=enum.AssetType.GEOTIFF, # GEOTIFF by default, can also be THUMBNAIL
                     bucket='my-bucket',                # the name of your bucket
                     region='us-central1',              # region your bucket is located in
                     save_directory='/')                # directory in which assets will be copied, <stac_id>.<file_ext>

# create the subscription, returning its id
sub_id = client_ex.subscribe_ex(request, destination=dst)
print(sub_id)

# retrieve all of your current subscriptions, viewing the most recent
subs = client_ex.subscriptions_ex()
most_recent_sub = subs[len(subs) - 1]
print(most_recent_sub)

# deactivate a subscription, pausing all deliveries
client_ex.unsubscribe_ex(sub_id=sub_id)

# reactivate a subscription, resuming all deliveries
client_ex.resubscribe_ex(sub_id=sub_id)


### AWS Delivery

The destination bucket preparation process for AWS is slightly more involved. We're ultimately going to create an AWS
IAM Role (e.g. `NSLDelivery`) to be assumed by NSL's AWS account,
and an accompanying policy giving NSL's AWS account access to the destination bucket (e.g. `my-bucket`).
  
1. In your AWS account where the bucket is located, browse to IAM service.
  
2. Choose Roles -> "Create role". Choose "Another AWS account", providing NSL's AWS account ID: `275780812376`.
  
3. Click next to Permissions
  
4. Click "Create Policy", select the tab "JSON" and adjust the template below, which will whitelist actions NSL can
perform on your chosen destination bucket.
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "VisualEditor0",
      "Effect": "Allow",
      "Action": "s3:ListBucket",
      "Resource": "arn:aws:s3:::my-bucket"
    },
    {
      "Sid": "VisualEditor1",
      "Effect": "Allow",
      "Action": [
        "s3:PutObject",
        "s3:GetObject",
        "s3:DeleteObject"
      ],
      "Resource": "arn:aws:s3:::my-bucket/*"
    }
  ]
}
```
  
5. Click "Next: Tags", adding any optional tags.
  
6. Name this policy (e.g. `NSLBucketAccess`), and finally click "Create policy" to finish creating the policy.
  
7. Go back to Roles, refresh, and then select the `NSLBucketAccess` policy you just created.
  
8. Now we're back to finalizing the creation of the Role, so give it a meaningful name, e.g. `NSLBucketDeliverer`.

9. Contact us so we can update our AWS account to assume this role and begin deliveries. 

Finally, lets create a `BaseDestination` object and finish creating our subscription:

In [ ]:
# creates a destination object to describe how assets should be delivered
dst = AWSDestination(asset_type=enum.AssetType.GEOTIFF, # GEOTIFF by default, can also be THUMBNAIL
                     role_arn='arn:aws:iam::<YOUR_ACCESS_KEY_ID>:role/NSLBucketDeliverer',
                     bucket='my-bucket',                # the name of your bucket
                     region='us-central1',              # region your bucket is located in
                     save_directory='/')                # directory in which assets will be copied, <stac_id>.<file_ext>

# create the subscription, returning its id
sub_id = client_ex.subscribe_ex(request, destination=dst)
print(sub_id)

# retrieve all of your current subscriptions, viewing the most recent
subs = client_ex.subscriptions_ex()
most_recent_sub = subs[len(subs) - 1]
print(most_recent_sub)

# deactivate a subscription, pausing all deliveries
client_ex.unsubscribe_ex(sub_id=sub_id)

# reactivate a subscription, resuming all deliveries
client_ex.resubscribe_ex(sub_id=sub_id)